# GridSearch + Validation croisée + Hyperparamètres (avec pipeline)

## Setup (identique)

In [1]:
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

X, y = load_wine(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## A) “Validation set” avec pipeline (hold-out) — pour mimer le pas à pas

Même logique que ton avant : on sépare un train/val, on choisit l’hyperparamètre sur val, puis on évalue sur test.
La différence : le scaling est encapsulé.


In [2]:
from sklearn.model_selection import train_test_split

X_subtrain, X_val, y_subtrain, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train
)

k_values = range(1, 31)
best_k, best_val = None, -1

for k in k_values:
    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("model", KNeighborsClassifier(n_neighbors=k))
    ])
    pipe.fit(X_subtrain, y_subtrain)
    val_acc = pipe.score(X_val, y_val)

    if val_acc > best_val:
        best_val = val_acc
        best_k = k

print("Meilleur k (val):", best_k, "| Val acc:", round(best_val, 3))

# modèle final -> évalué sur test une seule fois
final_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", KNeighborsClassifier(n_neighbors=best_k))
])
final_pipe.fit(X_train, y_train)
print("Test acc:", round(final_pipe.score(X_test, y_test), 3))

Meilleur k (val): 7 | Val acc: 1.0
Test acc: 1.0


## B) GridSearchCV + CV avec pipeline (la vraie méthode propre)

### 1) Définir la CV (comme avant)

In [3]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### 2) KNN : GridSearch sur n_neighbors

In [4]:
pipe_knn = Pipeline([
    ("scaler", StandardScaler()),
    ("model", KNeighborsClassifier())
])

param_grid_knn = {
    "model__n_neighbors": list(range(1, 31))
}

grid_knn = GridSearchCV(
    estimator=pipe_knn,
    param_grid=param_grid_knn,
    cv=cv,
    scoring="accuracy",
    n_jobs=-1
)

grid_knn.fit(X_train, y_train)

print("KNN best params:", grid_knn.best_params_)
print("KNN best CV acc:", round(grid_knn.best_score_, 3))
print("KNN test acc:", round(grid_knn.score(X_test, y_test), 3))

KNN best params: {'model__n_neighbors': 29}
KNN best CV acc: 0.979
KNN test acc: 0.972


La clé model__n_neighbors signifie : hyperparamètre n_neighbors de l’étape model dans le pipeline.

### 3) Logistic Regression : GridSearch sur C


In [5]:
pipe_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=5000, solver="lbfgs"))
])

param_grid_lr = {
    "model__C": [0.01, 0.1, 1, 10, 100]
}

grid_lr = GridSearchCV(
    estimator=pipe_lr,
    param_grid=param_grid_lr,
    cv=cv,
    scoring="accuracy",
    n_jobs=-1
)

grid_lr.fit(X_train, y_train)

print("LR best params:", grid_lr.best_params_)
print("LR best CV acc:", round(grid_lr.best_score_, 3))
print("LR test acc:", round(grid_lr.score(X_test, y_test), 3))


LR best params: {'model__C': 1}
LR best CV acc: 0.979
LR test acc: 0.972


## C) make_pipeline (raccourci) — même résultat, moins verbeux

In [6]:
pipe_knn_fast = make_pipeline(StandardScaler(), KNeighborsClassifier())
grid_knn_fast = GridSearchCV(
    pipe_knn_fast,
    {"kneighborsclassifier__n_neighbors": list(range(1, 31))},
    cv=cv,
    scoring="accuracy",
    n_jobs=-1
)
grid_knn_fast.fit(X_train, y_train)

print("Best params (make_pipeline):", grid_knn_fast.best_params_)

Best params (make_pipeline): {'kneighborsclassifier__n_neighbors': 29}


Avec make_pipeline, les noms d’étapes sont auto (kneighborsclassifier, etc.). C’est pratique, mais moins lisible que ('model', ...).

## D) Mini “multi-modèles” : comparer plusieurs pipelines proprement

In [7]:
pipelines = {
    "KNN": (pipe_knn, param_grid_knn),
    "LogReg": (pipe_lr, param_grid_lr)
}

results = {}
for name, (pipe, grid) in pipelines.items():
    gs = GridSearchCV(pipe, grid, cv=cv, scoring="accuracy", n_jobs=-1)
    gs.fit(X_train, y_train)
    results[name] = {
        "best_params": gs.best_params_,
        "cv_acc": gs.best_score_,
        "test_acc": gs.score(X_test, y_test)
    }

results

{'KNN': {'best_params': {'model__n_neighbors': 29},
  'cv_acc': np.float64(0.9788177339901478),
  'test_acc': 0.9722222222222222},
 'LogReg': {'best_params': {'model__C': 1},
  'cv_acc': np.float64(0.9790640394088669),
  'test_acc': 0.9722222222222222}}

## (Optionnel :Affichage des pipelines et meilleurs params de chaque modèle)

## Mini laïus final (récap modules / “qui fait quoi”)

### Pipeline : enchaîne des étapes séquentielles (ex: scaler → modèle).  
✅ Garantit que le preprocessing est refait correctement à chaque .fit() (donc dans chaque fold de CV).  

### make_pipeline : raccourci de Pipeline.  
✅ Plus rapide à écrire, ❗ noms d’étapes automatiques (moins clair pour expliquer __).  

### StratifiedKFold : comment on découpe les folds en CV.  
✅ “Stratified” conserve les proportions de classes dans chaque fold (important en classification).  

### GridSearchCV : quoi on teste (une grille d’hyperparamètres) + comment on évalue (via CV).  
✅ Il entraîne plein de modèles, compare les scores CV, garde le meilleur (best_estimator_).  

### Validation set vs CV :  

- Validation set = 1 split → simple mais instable (dépend du hasard).  

- CV = plusieurs splits → score plus robuste, tuning plus fiable.  

### Le test set : jamais utilisé pour choisir des hyperparamètres.  
✅ Sert uniquement à l’évaluation finale “comme en prod”.  